In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\bianc\AppData\Local\Temp\ipykernel_5792\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development.

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development.
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1000 non-null   object
 1   label   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [5]:
data.nunique()


text     946
label      2
dtype: int64

### Let's divide the training and test set into two partitions

In [6]:
# Your code
data_train = data[:900]
data_val = data[900:]

## Data Preprocessing

In [7]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [8]:
# Your code

import re

#print(data.head(5))

patterns_html = [
    re.compile(r'<script\b.[^>]*>.*?</script>', flags=re.DOTALL | re.IGNORECASE), # remove script blocks
    re.compile(r'<style\b[^>]*>.*?</style>', flags=re.DOTALL | re.IGNORECASE), # remove style blocks
    re.compile(r'\s+style\s*=\s*(".*?"|\'.*?\'|[^\s>]+)', flags=re.IGNORECASE), # remove style attributes
    re.compile(r'<!--.*?-->', flags=re.DOTALL), # remove HTML comments
    re.compile(r'<[^>]+>') # remove any other HTML tags
    ]

def clean_html(text):
    for pattern in patterns_html:
        text = pattern.sub(" ", text)

    return " ".join(text.split())

data["text_clean"] = data["text"].apply(clean_html)

#data.head(2)

In [9]:
data.head()

,text,label,text_clean
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL..."
1,Will do.,0,Will do.
2,Nora--Cheryl has emailed dozens of memos about...,0,Nora--Cheryl has emailed dozens of memos about...
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,Dear Sir=2FMadam=2C I know that this proposal ...
4,fyi,0,fyi


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters

- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [10]:
# remove all special characters
pattern_text1 = re.compile(r'[\`\!\@\#\$\%\^\&\*\(\)\_\+\-\=\[\]\{\}\;\'\:\"\\\|\,\.\<\>\/\?\~]+')

def clean_text1(text):
    text = pattern_text1.sub(" ", text)
    return " ".join(text.split())

data["text_clean"] = data["text_clean"].apply(clean_text1)

data.head()

,text,label,text_clean
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,DEAR SIR STRICTLY A PRIVATE BUSINESS PROPOSAL ...
1,Will do.,0,Will do
2,Nora--Cheryl has emailed dozens of memos about...,0,Nora Cheryl has emailed dozens of memos about ...
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,Dear Sir 2FMadam 2C I know that this proposal ...
4,fyi,0,fyi


In [11]:
# remove numbers
pattern_text2 = re.compile(r'\d+')

def clean_text2(text):
    text = pattern_text2.sub(" ", text)
    return " ".join(text.split())

data["text_clean"] = data["text_clean"].apply(clean_text2)

data.head(7)

,text,label,text_clean
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,DEAR SIR STRICTLY A PRIVATE BUSINESS PROPOSAL ...
1,Will do.,0,Will do
2,Nora--Cheryl has emailed dozens of memos about...,0,Nora Cheryl has emailed dozens of memos about ...
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,Dear Sir FMadam C I know that this proposal mi...
4,fyi,0,fyi
5,sure -- bottom line - you need a special secur...,0,sure bottom line you need a special security c...
6,"Dear Sir,I am Engr. Ugo Nzego with the Enginee...",1,Dear Sir I am Engr Ugo Nzego with the Engineer...


In [12]:
# remove all single characters
pattern_text3 = re.compile(r'\b[a-zA-Z]\b')

def clean_text3(text):
    text = pattern_text3.sub(" ", text)
    return " ".join(text.split())

data["text_clean"] = data["text_clean"].apply(clean_text3)

data.head(7)

,text,label,text_clean
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,DEAR SIR STRICTLY PRIVATE BUSINESS PROPOSAL AM...
1,Will do.,0,Will do
2,Nora--Cheryl has emailed dozens of memos about...,0,Nora Cheryl has emailed dozens of memos about ...
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,Dear Sir FMadam know that this proposal might ...
4,fyi,0,fyi
5,sure -- bottom line - you need a special secur...,0,sure bottom line you need special security cod...
6,"Dear Sir,I am Engr. Ugo Nzego with the Enginee...",1,Dear Sir am Engr Ugo Nzego with the Engineerin...


In [13]:
# substitute multiple spaces with single space
pattern_text4 = re.compile(r'\s{2,}')

def clean_text4(text):
    text = pattern_text4.sub(" ", text)
    return " ".join(text.split())

data["text_clean"] = data["text_clean"].apply(clean_text4)

data.head(7)

,text,label,text_clean
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,DEAR SIR STRICTLY PRIVATE BUSINESS PROPOSAL AM...
1,Will do.,0,Will do
2,Nora--Cheryl has emailed dozens of memos about...,0,Nora Cheryl has emailed dozens of memos about ...
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,Dear Sir FMadam know that this proposal might ...
4,fyi,0,fyi
5,sure -- bottom line - you need a special secur...,0,sure bottom line you need special security cod...
6,"Dear Sir,I am Engr. Ugo Nzego with the Enginee...",1,Dear Sir am Engr Ugo Nzego with the Engineerin...


In [14]:
# remove prefixed 'b
pattern_text5 = re.compile(r'^b\'')

def clean_text5(text):
    text = pattern_text5.sub("", text)
    return " ".join(text.split())

data["text_clean"] = data["text_clean"].apply(clean_text5)

data.head(7)

,text,label,text_clean
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,DEAR SIR STRICTLY PRIVATE BUSINESS PROPOSAL AM...
1,Will do.,0,Will do
2,Nora--Cheryl has emailed dozens of memos about...,0,Nora Cheryl has emailed dozens of memos about ...
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,Dear Sir FMadam know that this proposal might ...
4,fyi,0,fyi
5,sure -- bottom line - you need a special secur...,0,sure bottom line you need special security cod...
6,"Dear Sir,I am Engr. Ugo Nzego with the Enginee...",1,Dear Sir am Engr Ugo Nzego with the Engineerin...


In [15]:
# Your code

# preprocessing: lowercase
data['text_clean'] = data['text_clean'].str.lower()
data.head()

,text,label,text_clean
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,dear sir strictly private business proposal am...
1,Will do.,0,will do
2,Nora--Cheryl has emailed dozens of memos about...,0,nora cheryl has emailed dozens of memos about ...
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,dear sir fmadam know that this proposal might ...
4,fyi,0,fyi


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [16]:
# Your code
#import nltk
#nltk.download('punkt_tab') # try to use this is you need some package
#nltk.download('punkt_tab', download_dir='/root/nltk_data') # Google colab: try to use this is you need some package

from nltk.tokenize import word_tokenize

# preprocessing: tokenize, lowercase, remove punctuation
def tokenizer_and_remove_punctuation(row):
    tokens = word_tokenize(row['text_clean'])
    return [word.lower() for word in tokens if word.isalpha()]

data['text_token'] = data.apply(tokenizer_and_remove_punctuation, axis=1)

data.head()

,text,label,text_clean,text_token
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,dear sir strictly private business proposal am...,"[dear, sir, strictly, private, business, propo..."
1,Will do.,0,will do,"[will, do]"
2,Nora--Cheryl has emailed dozens of memos about...,0,nora cheryl has emailed dozens of memos about ...,"[nora, cheryl, has, emailed, dozens, of, memos..."
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,dear sir fmadam know that this proposal might ...,"[dear, sir, fmadam, know, that, this, proposal..."
4,fyi,0,fyi,[fyi]


## Now let's work on removing stopwords
Remove the stopwords.

In [17]:
# Your code
#import nltk
#nltk.download()   # try to use this is you need some package
#nltk.download('stopwords', download_dir='/root/nltk_data') # Google colab: try to use this is you need some package

from nltk.corpus import stopwords
print (stopwords.words('english'))

# remove stopwords
def remove_stopwords(row):
    return list(set(row['text_token']).difference(stopwords.words()))

# data train: remove stopwords for tokenization
data['preprocessed_text'] = data.apply(remove_stopwords, axis=1)

data.head()

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

,text,label,text_clean,text_token,preprocessed_text
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,dear sir strictly private business proposal am...,"[dear, sir, strictly, private, business, propo...","[local, plc, appropriate, telephone, return, u..."
1,Will do.,0,will do,"[will, do]",[]
2,Nora--Cheryl has emailed dozens of memos about...,0,nora cheryl has emailed dozens of memos about ...,"[nora, cheryl, has, emailed, dozens, of, memos...","[dozens, thx, haiti, inchrono, order, print, m..."
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,dear sir fmadam know that this proposal might ...,"[dear, sir, fmadam, know, that, this, proposal...","[desperate, wish, foriegn, equity, beneficiary..."
4,fyi,0,fyi,[fyi],[fyi]


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [ ]:
# Your code
from collections import Counter

all_counter = Counter()

for tokens in data['preprocessed_text']:
    all_counter.update(tokens)

all_counter.most_common(10)

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


## Extra features

In [21]:
data_train = data[:900]
data_val = data[900:]

In [22]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x))

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x))

data_train.head()

C:\Users\bianc\AppData\Local\Temp\ipykernel_5792\2256358202.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
C:\Users\bianc\AppData\Local\Temp\ipykernel_5792\2256358202.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
C:\Users\bianc\AppData\Local\Temp\ipykernel_5792\2256358202.py:7: SettingWithCopyWarning: 
A valu

,text,label,text_clean,text_token,preprocessed_text,money_mark,suspicious_words,text_len
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,dear sir strictly private business proposal am...,"[dear, sir, strictly, private, business, propo...","[local, plc, appropriate, telephone, return, u...",NaN,NaN,126
1,Will do.,0,will do,"[will, do]",[],NaN,NaN,0
2,Nora--Cheryl has emailed dozens of memos about...,0,nora cheryl has emailed dozens of memos about ...,"[nora, cheryl, has, emailed, dozens, of, memos...","[dozens, thx, haiti, inchrono, order, print, m...",NaN,NaN,15
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,dear sir fmadam know that this proposal might ...,"[dear, sir, fmadam, know, that, this, proposal...","[desperate, wish, foriegn, equity, beneficiary...",NaN,NaN,127
4,fyi,0,fyi,[fyi],[fyi],NaN,NaN,1


## How would work the Bag of Words with Count Vectorizer concept?

In [28]:
# Your code

from sklearn.feature_extraction.text import CountVectorizer

bow_vect = CountVectorizer(max_features=1000)

X_train = bow_vect.fit_transform(data_train['preprocessed_text'].apply(lambda x: " ".join(x))).toarray()

X_val = bow_vect.transform(data_val['preprocessed_text'].apply(lambda x: " ".join(x))).toarray()

y_train = data_train['label'].values
y_val = data_val['label'].values


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvector=TfidfVectorizer(ngram_range=(2,2))

X_train_tf = tfidfvector.fit_transform(data_train['preprocessed_text'].apply(lambda x: " ".join(x))).toarray()

X_val_tf = tfidfvector.transform(data_val['preprocessed_text'].apply(lambda x: " ".join(x))).toarray()

## And the Train a Classifier?

In [29]:
# Your code

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# create the model
reg = LogisticRegression(max_iter=200, random_state=100)

# fit the model with the training set
reg.fit(X_train, y_train)

# evaluate the model with validation set
y_val_pred = reg.predict(X_val)

# display accuracy
accuracy = accuracy_score(y_val, y_val_pred)
print("Accuracy:", accuracy)
print(classification_report(y_val, y_val_pred))

Accuracy: 0.96
              precision    recall  f1-score   support

           0       0.95      0.98      0.97        62
           1       0.97      0.92      0.95        38

    accuracy                           0.96       100
   macro avg       0.96      0.95      0.96       100
weighted avg       0.96      0.96      0.96       100



In [32]:
# Your code

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# create the model
reg2 = LogisticRegression(max_iter=200, random_state=100)

# fit the model with the training set
reg2.fit(X_train_tf, y_train)

# evaluate the model with validation set
y_val_pred2 = reg2.predict(X_val_tf)

# display accuracy
accuracy = accuracy_score(y_val, y_val_pred2)
print("Accuracy:", accuracy)
print(classification_report(y_val, y_val_pred2))

Accuracy: 0.9
              precision    recall  f1-score   support

           0       0.86      1.00      0.93        62
           1       1.00      0.74      0.85        38

    accuracy                           0.90       100
   macro avg       0.93      0.87      0.89       100
weighted avg       0.91      0.90      0.90       100



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [33]:
# Your code

from sklearn.naive_bayes import MultinomialNB

# create the model
model_nb = MultinomialNB()

# fit the model with the training set
model_nb.fit(X_train, y_train)

# evaluate the model with validation set
y_val_pred3 = model_nb.predict(X_val)

# display accuracy
accuracy = accuracy_score(y_val, y_val_pred3)
print("Accuracy:", accuracy)
print(classification_report(y_val, y_val_pred3))

Accuracy: 0.95
              precision    recall  f1-score   support

           0       0.95      0.97      0.96        62
           1       0.95      0.92      0.93        38

    accuracy                           0.95       100
   macro avg       0.95      0.94      0.95       100
weighted avg       0.95      0.95      0.95       100



In [35]:
# Your code

# create the model
model_nb2 = MultinomialNB()

# fit the model with the training set
model_nb2.fit(X_train_tf, y_train)

# evaluate the model with validation set
y_val_pred4 = model_nb2.predict(X_val_tf)

# display accuracy
accuracy = accuracy_score(y_val, y_val_pred4)
print("Accuracy:", accuracy)
print(classification_report(y_val, y_val_pred4))

Accuracy: 0.93
              precision    recall  f1-score   support

           0       0.92      0.97      0.94        62
           1       0.94      0.87      0.90        38

    accuracy                           0.93       100
   macro avg       0.93      0.92      0.92       100
weighted avg       0.93      0.93      0.93       100

